In [1]:
from fastai.vision.all import *
import fastai
import timm
import torch
from sklearn.metrics import cohen_kappa_score,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

device = 1

torch.cuda.set_device(device)



In [2]:
results = []
resultsTTA = []
for i in range(1,6):
    df = pd.read_csv('../trainValid'+str(i)+'.csv')
    
    def is_valid(path):
        name = path[1]
        return (df[df['nombre_foto']==name])['set'].values[0]=='validation'
    
    
    db = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 splitter=FuncSplitter(is_valid),
                 get_x = ColReader(1,pref="../amd/"),
                 get_y=ColReader(3),
                 item_tfms = [Resize(384)], # CropPad(200,200)
                 batch_tfms=[*aug_transforms(size=224, min_scale=0.75,do_flip=True,flip_vert=True,
                      max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                             Normalize.from_stats(*imagenet_stats)])
    
    callbacks = [
        #ShowGraphCallback(),
        EarlyStoppingCallback(patience=5),
        SaveModelCallback(fname='resnet224_'+str(i),monitor='f1_score'),
        #ReduceLROnPlateau(patience=2)
    ]
    
    dls = db.dataloaders(df.values,bs=128,num_workers=0)
    
    learn = cnn_learner(dls,resnet50,
                metrics=[accuracy,Precision(),Precision(pos_label=0),Recall(),Recall(pos_label=0),F1Score(),RocAucBinary(),CohenKappa()],
                cbs=callbacks,
                loss_func= FocalLossFlat()).to_fp16()
    
    learn.fine_tune(100,base_lr=1e-3)
    
    
    dfTest = pd.read_csv('../test'+str(i)+'.csv')
    dfTest.set = dfTest.set.apply(lambda x : 'test')
    dfTest = pd.concat([df[df.set=='validation'],dfTest])
    
    def is_test(path):
        name = path[1]
        return (dfTest[dfTest['nombre_foto']==name])['set'].values[0]=='test'
    
    dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 splitter=FuncSplitter(is_test),
                 get_x = ColReader(1,pref="../amd/"),
                 get_y=ColReader(3),
                 item_tfms = [Resize(384)], # CropPad(200,200)
                 batch_tfms=[*aug_transforms(size=224, min_scale=0.75,do_flip=True,flip_vert=True,
                      max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                             Normalize.from_stats(*imagenet_stats)])
    dlsTest = dbTest.dataloaders(dfTest.values,bs=64,num_workers=0)
    learn.dls = dlsTest
    res = learn.validate()
    results.append(res)
    resTTA = learn.tta()
    accuracy = accuracy_score(resTTA[1],np.argmax(resTTA[0],axis=1).tolist())
    precision = precision_score(resTTA[1],np.argmax(resTTA[0],axis=1).tolist())
    npv = precision_score(resTTA[1],np.argmax(resTTA[0],axis=1).tolist(),pos_label=0)
    recall = recall_score(resTTA[1],np.argmax(resTTA[0],axis=1).tolist())
    specificity = recall_score(resTTA[1],np.argmax(resTTA[0],axis=1).tolist(),pos_label=0)
    f1score = f1_score(resTTA[1],np.argmax(resTTA[0],axis=1).tolist())
    roc = roc_auc_score(resTTA[1],resTTA[0][:,1])
    cohen = cohen_kappa_score(resTTA[1],np.argmax(resTTA[0],axis=1).tolist(),weights='quadratic')
    resultsTTA.append([accuracy,precision,npv,recall,specificity,f1score,roc,cohen])
    torch.cuda.empty_cache()
    
print('Results:')
print(np.mean(results,axis=0)[1:])
print(np.std(results,axis=0)[1:])

print('Results TTA:')
print(np.mean(resultsTTA,axis=0))
print(np.std(resultsTTA,axis=0))  

epoch,train_loss,valid_loss,accuracy,precision_score,precision_score,recall_score,recall_score,f1_score,roc_auc_score,cohen_kappa_score,time
0,0.683916,0.427578,0.769231,0.835616,0.723404,0.675900,0.864407,0.747320,0.859360,0.539288,01:11


Better model found at epoch 0 with f1_score value: 0.7473200612557427.


epoch,train_loss,valid_loss,accuracy,precision_score,precision_score,recall_score,recall_score,f1_score,roc_auc_score,cohen_kappa_score,time
0,0.437088,0.252144,0.805594,0.872483,0.757794,0.720222,0.892655,0.789074,0.894709,0.611821,01:15
1,0.369245,0.197533,0.827972,0.843931,0.813008,0.808864,0.847458,0.826025,0.905074,0.656052,01:14
2,0.313803,0.205145,0.827972,0.902027,0.775656,0.739612,0.918079,0.812785,0.911510,0.656523,01:13
3,0.271520,0.165322,0.848951,0.856338,0.841667,0.842105,0.855932,0.849162,0.916898,0.697923,01:10
4,0.240917,0.162209,0.843357,0.865103,0.823529,0.817175,0.870056,0.840456,0.925067,0.686855,01:11
5,0.211208,0.139876,0.841958,0.882716,0.808184,0.792244,0.892655,0.835036,0.928901,0.684206,01:11
6,0.179979,0.141976,0.847552,0.898734,0.807018,0.786704,0.909605,0.838996,0.931722,0.695451,01:13
7,0.159153,0.123308,0.860140,0.855586,0.864943,0.869806,0.850282,0.862637,0.934930,0.720207,01:13
8,0.139822,0.111222,0.861538,0.876437,0.847411,0.844875,0.878531,0.860367,0.940991,0.723149,01:13
9,0.119583,0.110536,0.868531,0.903323,0.838542,0.828255,0.909605,0.864162,0.939113,0.737254,01:12


Better model found at epoch 0 with f1_score value: 0.7890743550834597.
Better model found at epoch 1 with f1_score value: 0.8260254596888261.
Better model found at epoch 3 with f1_score value: 0.8491620111731844.
Better model found at epoch 7 with f1_score value: 0.8626373626373626.
Better model found at epoch 9 with f1_score value: 0.8641618497109828.
Better model found at epoch 11 with f1_score value: 0.8753462603878115.
Better model found at epoch 16 with f1_score value: 0.8860398860398859.
No improvement since epoch 12: early stopping


Better model found at epoch 0 with f1_score value: 0.9487891287385278.


epoch,train_loss,valid_loss,None,precision_score,precision_score,recall_score,recall_score,f1_score,roc_auc_score,cohen_kappa_score,time


TypeError: 'numpy.float64' object is not callable